In [34]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
dataset

Using custom data configuration default
Found cached dataset rotten_tomatoes (/Users/alyssahuang/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [35]:
dataset['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [36]:
def add_end_of_text(example):
    example['text'] =  example['text'] + '<|endoftext|>'
    return example

dataset = dataset.remove_columns(['label'])
dataset = dataset.map(add_end_of_text)

  0%|          | 0/8530 [00:00<?, ?ex/s]

  0%|          | 0/1066 [00:00<?, ?ex/s]

  0%|          | 0/1066 [00:00<?, ?ex/s]

In [38]:
dataset['train']['text'][:10]
dataset['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .<|endoftext|>'}

In [39]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [40]:
sequence = ("This tokenizer is being applied in CS197 at"
            "Harvard.<|endoftext|>")
tokens = tokenizer.tokenize(sequence)
print(tokens)

['this', 'token', '##izer', 'is', 'being', 'applied', 'in', 'cs', '##19', '##7', 'at', '##har', '##vard', '.', '<', '|', 'end', '##oft', '##ex', '##t', '|', '>']


In [19]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2023, 19204, 17629, 2003, 2108, 4162, 1999, 20116, 16147, 2581, 2012, 8167, 25911, 1012, 1026, 1064, 2203, 15794, 10288, 2102, 1064, 1028]


In [20]:
sequence = ("This tokenizer is being applied in CS197 at"
            "Harvard.<|endoftext|>")
tokenizer(sequence)

{'input_ids': [101, 2023, 19204, 17629, 2003, 2108, 4162, 1999, 20116, 16147, 2581, 2012, 8167, 25911, 1012, 1026, 1064, 2203, 15794, 10288, 2102, 1064, 1028, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [41]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"]
)

print(dataset["train"][0])
print(tokenized_datasets["train"][0])

#0:   0%|          | 0/3 [00:00<?, ?ba/s]

#1:   0%|          | 0/3 [00:00<?, ?ba/s]

#2:   0%|          | 0/3 [00:00<?, ?ba/s]

#3:   0%|          | 0/3 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .<|endoftext|>'}
{'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 1026, 1064, 2203, 15794, 10288, 2102, 1064, 1028, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [42]:
block_size = 128

def group_texts(examples):
    # repeat concatenation for input_ids and other keys
    concatenated_examples = {k: sum(examples[k], []) for k in
                            examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size

    # populate each of input_ids and other keys 
    result = {
        k: [t[i : i + block_size] for i in range(0,
            total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # add labels because we'll need it as the output
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/3 [00:00<?, ?ba/s]

#2:   0%|          | 0/3 [00:00<?, ?ba/s]

#3:   0%|          | 0/3 [00:00<?, ?ba/s]

#1:   0%|          | 0/3 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [43]:
print(lm_datasets['train']['input_ids'][0])

[101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 1026, 1064, 2203, 15794, 10288, 2102, 1064, 1028, 102, 101, 1996, 9882, 2135, 9603, 13633, 1997, 1000, 1996, 2935, 1997, 1996, 7635, 1000, 11544, 2003, 2061, 4121, 2008, 1037, 5930, 1997, 2616, 3685, 23613, 6235, 2522, 1011, 3213, 1013, 2472, 2848, 4027, 1005, 1055, 4423, 4432, 1997, 1046, 1012, 1054, 1012, 1054, 1012, 23602, 1005, 1055, 2690, 1011, 3011, 1012, 1026, 1064, 2203, 15794, 10288, 2102, 1064, 1028, 102, 101, 4621, 2021, 2205, 1011, 8915, 23267, 16012, 24330, 1026, 1064, 2203, 15794]


In [44]:
tokenizer.decode(lm_datasets['train']['input_ids'][0])

'[CLS] the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger, jean - claud van damme or steven segal. < | endoftext | > [SEP] [CLS] the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co - writer / director peter jackson\'s expanded vision of j. r. r. tolkien\'s middle - earth. < | endoftext | > [SEP] [CLS] effective but too - tepid biopic < | endoft'

In [45]:
small_train_dataset = \
    lm_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = \
    lm_datasets["validation"].shuffle(seed=42).select(range(100))

In [47]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
training_args = TrainingArguments(
    output_dir="./models"
    f"{model_checkpoint}-rotten-tomatoes",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

trainer.train()

Cloning https://huggingface.co/alyssahuang02/modelsbert-base-uncased-rotten-tomatoes into local empty directory.
/Users/alyssahuang/miniconda/envs/lec4/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alyssahuang02. Use `wandb login --re

  0%|          | 0/39 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 5.949063777923584, 'eval_runtime': 15.7345, 'eval_samples_per_second': 6.355, 'eval_steps_per_second': 0.826, 'epoch': 1.0}


...

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 4.895145416259766, 'eval_runtime': 14.3601, 'eval_samples_per_second': 6.964, 'eval_steps_per_second': 0.905, 'epoch': 3.0}
{'train_runtime': 227.7952, 'train_samples_per_second': 1.317, 'train_steps_per_second': 0.171, 'train_loss': 6.385534433218149, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=6.385534433218149, metrics={'train_runtime': 227.7952, 'train_samples_per_second': 1.317, 'train_steps_per_second': 0.171, 'train_loss': 6.385534433218149, 'epoch': 3.0})

In [49]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

Perplexity: 133.64


In [50]:
tokenizer.save_pretrained(f"{model_checkpoint}-rotten-tomatoes")
model.push_to_hub(f"{model_checkpoint}-rotten-tomatoes")

tokenizer config file saved in bert-base-uncased-rotten-tomatoes/tokenizer_config.json
Special tokens file saved in bert-base-uncased-rotten-tomatoes/special_tokens_map.json
Configuration saved in bert-base-uncased-rotten-tomatoes/config.json
Model weights saved in bert-base-uncased-rotten-tomatoes/pytorch_model.bin
Uploading the following files to alyssahuang02/bert-base-uncased-rotten-tomatoes: config.json,pytorch_model.bin


In [54]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(f"./{model_checkpoint}-rotten-tomatoes")
tokenizer = AutoTokenizer.from_pretrained(f"./{model_checkpoint}-rotten-tomatoes")


loading configuration file ./bert-base-uncased-rotten-tomatoes/config.json
Model config BertConfig {
  "_name_or_path": "./bert-base-uncased-rotten-tomatoes",
  "architectures": [
    "BertLMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./bert-base-uncased-rotten-tomatoes/pytorch_model.bin
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
All model checkpoint weights were used when initializing BertLMHea

wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
# start_text = ("A speedrun is a playthrough of a video game, \
# or section of a video game, with the goal of \
# completing it as fast as possible. Speedruns \
# often follow planned routes, which may incorporate sequence \
# breaking, and might exploit glitches that allow sections to \
# be skipped or completed more quickly than intended. ")

# prompt = "What is the"
# inputs = tokenizer(
#      start_text + prompt,
#      add_special_tokens=False,
#      return_tensors="pt")["input_ids"]

# prompt_length = len(tokenizer.decode(inputs[0]))
# outputs = model.generate(
#      inputs,
#      max_length=100,
#      do_sample=True,
#      top_k=50,
#      top_p=0.95,
#      temperature=0.9,
#      num_return_sequences=3)

# generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1:]

# print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A speedrun is a playthrough of a video game, or section of a video game, with the goal of completing it as fast as possible. Speedruns often follow planned routes, which may incorporate sequence breaking, and might exploit glitches that allow sections to be skipped or completed more quickly than intended. What is the goal of Speedrun?<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
